# Appendix 4 - Server-based acquisition

In this appendix, it describes the process of implementing an acquisition server based on Raspberry Pi, the main goal of using Raspberry is to get a full controlled system. The operating system chosen for this purpose is [Arch Linux ARM](https://archlinuxarm.org/).

## Step 1 - Install Arch Linux ARM

The process of installatión is very simple, there is a [guide](https://archlinuxarm.org/platforms/armv8/broadcom/raspberry-pi-4) on the official site.  
After a complete system update:

In [ ]:
# pacman-key --init
# pacman-key --populate archlinuxarm
# pacman -Suy

## Step 2 - Install dependencies

The `kafka` installation can be done by using [yay](https://github.com/Jguer/yay):

In [ ]:
$ yay -S kafka

And the python dependencies by:

In [ ]:
# pacman -S python python-pip gcc cython hdf5
# pip install openbci-stream

:::note

Arch Linux ARM could be a bit fussy about install `scipy` from `pip`, the best option here is use the default package manager:  
`pacman -S python-scipy --overwrite='/usr/lib/python3.8/site-packages/*' --overwrite='/usr/bin/*'`  

:::

## Step 3 - Configure Kafka

Remove the option `-XX:+UseCompressedOops` from `/usr/lib/systemd/system/kafka.service`, open the file and edit it, you can use `vim` (not instaled by default) or `nano`:

In [ ]:
# vim /usr/lib/systemd/system/kafka.service
# systemctl daemon-reload

The other configuration can be done simply by executing the script `stream_configure_kafka.sh` included on `openbci-stream` package:

In [ ]:
# stream_configure_kafka.sh 

## Step 4 - Configure an access point

The main purpose of this guide is to configure the Raspberry as an access point, the next script will configure a WiFi network called `OpenBCI-Stream` with password `raspberrypi`:

In [ ]:
# stream_access_point.sh

:::note

So, if you are connected via WiFi, this connection will be lost after restart the Raspberry, you may wish to connect using other methods (Ethernet, USB) in ways to continue the configuration.  

:::

## Step 5 - Start acquisition daemons

In [ ]:
# stream_eeg systemd
# stream_rpyc systemd

The commands above will create the respective services for acquisition streaming and distributed access. And then, for initialize them:

In [ ]:
# systemctl enable stream_eeg stream_rpyc
# systemctl start stream_eeg stream_rpyc

## Step 6 - Test server

For a complete test, we will need an OpenBCI board connected, in any mode, Serial or WiFi (so be sure about configuring serial port permissions).

### WiFi Shield
For a WiFi shield, we must know the correct IP, usually this IP not change over time, so if we found it the first time is very likely that we can use the same in other opportunities.  
You can use [nmap](https://wiki.archlinux.org/index.php/nmap#Ping_scan) to scan devices:

In [ ]:
$ ssh alarm@192.168.1.1
$ nmap -sn 192.168.1.2/24

### Command line tests
So, we have `OpenBCI-Stream` installed on Raspberry, now we need a version installed locally, also we need to have [Kafka installed and running locally](02-kafka_configuration.ipynb).

In [ ]:
$ openbci_cli wifi --ip 192.168.1.113 --host 192.168.1.1 -c SAMPLE_RATE_250SPS

Here, `192.168.1.113` is the IP assigned from Raspberry to the WiFi module and `192.168.1.1` is the Raspberry IP itself. If everything is ok, a data stream will be printed on the terminal.

### Troubleshooting

  * **No streaming data:** Make sure that the WiFi module was still connected and the Serial devices have read and write permissions.
  * **Kafka no connection:** Kafka must be running on every single machine that wants to connect to the stream, there is no special configuring, the default is enough.
 